In [1]:
from lvq import *
from scipy.io import arff
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
from math import isnan

In [2]:
data = arff.loadarff('datasets/jm1.arff')
df = pd.DataFrame(data[0])
data = df.values
df.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,b'false'
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,b'true'
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51.0,10.0,8.0,1.0,17.0,36.0,112.0,86.0,13.0,b'true'
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129.0,29.0,28.0,2.0,17.0,135.0,329.0,271.0,5.0,b'true'
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28.0,1.0,6.0,0.0,11.0,16.0,76.0,50.0,7.0,b'true'


In [3]:
df['defects'] = pd.factorize(df['defects'])[0] + 1
df_norm = (df.iloc[:,:-1] - df.iloc[:,:-1].mean()) / (df.iloc[:,:-1].max() - df.iloc[:,:-1].min())
df_norm['defects'] = df['defects']
data = df_norm.values
np.random.shuffle(data)

In [4]:
prototypes = lvq_1(data, 200, 0.3, 10)
df_norm.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,-0.011892,-0.010550,-0.012195,-0.006487,-0.013398,-0.008318,0.895883,-0.030792,-0.049385,-0.001185,...,-0.008588,-0.002148,-0.005873,0.015084,-0.024276,-0.015158,-0.012345,-0.014958,-0.011991,1
1,-0.011921,-0.011402,-0.014634,-0.007485,-0.013433,-0.008322,0.665114,-0.031510,-0.049912,-0.001185,...,-0.008942,-0.005055,-0.008110,0.005824,-0.024763,-0.015353,-0.012382,-0.015024,-0.012476,2
2,0.008713,0.001391,-0.014634,0.004984,0.009905,0.005694,-0.065655,0.014664,0.046353,-0.000445,...,0.008763,0.021108,0.007550,0.005824,0.014166,0.018760,0.008098,0.013112,0.002070,2
3,0.043005,-0.007138,-0.014634,-0.002497,0.057530,0.045458,-0.057963,0.006893,0.395646,0.001202,...,0.036383,0.076341,0.052293,0.015084,0.014166,0.115252,0.048135,0.074350,-0.007627,2
4,-0.001459,-0.005006,-0.014634,-0.000004,0.001375,-0.000923,-0.057963,0.007204,0.009514,-0.000854,...,0.000619,-0.005055,0.003076,-0.003435,-0.000432,-0.000733,0.001456,0.001195,-0.005203,2


In [5]:
x_train = np.asarray(prototypes)[:,:-1]
y_train = np.asarray(prototypes)[:,-1]
x_test = np.asarray(data)[:,:-1]
y_test = np.asarray(data)[:,-1]

In [6]:
knn = KNeighborsClassifier(n_neighbors=1, n_jobs=4)
knn.fit(x_train, y_train)
predicts = knn.predict(x_test)
print(((predicts == y_test).sum()/len(predicts))*100)

81.22242647058823


In [7]:
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=4)
knn.fit(x_train, y_train)
predicts = knn.predict(x_test)
print(((predicts == y_test).sum()/len(predicts))*100)

75.9466911764706


In [8]:
prototypes = lvq_21(data, 200, 0.3, 10)
df_norm.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,-0.011892,-0.010550,-0.012195,-0.006487,-0.013398,-0.008318,0.895883,-0.030792,-0.049385,-0.001185,...,-0.008588,-0.002148,-0.005873,0.015084,-0.024276,-0.015158,-0.012345,-0.014958,-0.011991,1
1,-0.011921,-0.011402,-0.014634,-0.007485,-0.013433,-0.008322,0.665114,-0.031510,-0.049912,-0.001185,...,-0.008942,-0.005055,-0.008110,0.005824,-0.024763,-0.015353,-0.012382,-0.015024,-0.012476,2
2,0.008713,0.001391,-0.014634,0.004984,0.009905,0.005694,-0.065655,0.014664,0.046353,-0.000445,...,0.008763,0.021108,0.007550,0.005824,0.014166,0.018760,0.008098,0.013112,0.002070,2
3,0.043005,-0.007138,-0.014634,-0.002497,0.057530,0.045458,-0.057963,0.006893,0.395646,0.001202,...,0.036383,0.076341,0.052293,0.015084,0.014166,0.115252,0.048135,0.074350,-0.007627,2
4,-0.001459,-0.005006,-0.014634,-0.000004,0.001375,-0.000923,-0.057963,0.007204,0.009514,-0.000854,...,0.000619,-0.005055,0.003076,-0.003435,-0.000432,-0.000733,0.001456,0.001195,-0.005203,2


In [9]:
x_train = np.asarray(prototypes)[:,:-1]
y_train = np.asarray(prototypes)[:,-1]
x_test = np.asarray(data)[:,:-1]
y_test = np.asarray(data)[:,-1]

In [10]:
knn = KNeighborsClassifier(n_neighbors=1, n_jobs=4)
knn.fit(x_train, y_train)
predicts = knn.predict(x_test)
print(((predicts == y_test).sum()/len(predicts))*100)

81.19485294117646


In [11]:
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=4)
knn.fit(x_train, y_train)
predicts = knn.predict(x_test)
print(((predicts == y_test).sum()/len(predicts))*100)

80.88235294117648
